Importing Neccessary Libraries

In [1]:
import re
import os
import pandas as pd
from parse_football_stats import parse_football_stats
from top_n_outcomes_from_folder import top_n_outcomes_from_folder

                  Game       Team Score   Odds  Probability
0   Luton vs Sheff Utd       Draw   1-1    7/1     0.142857
1   Luton vs Sheff Utd      Luton   1-0    8/1     0.125000
2   Luton vs Sheff Utd      Luton   2-1    8/1     0.125000
3   Luton vs Sheff Utd      Luton   2-0   17/2     0.117647
4   Luton vs Sheff Utd       Draw   0-0   12/1     0.083333
5   Luton vs Sheff Utd  Sheff Utd   1-0   14/1     0.071429
6   Luton vs Sheff Utd      Luton   3-0   14/1     0.071429
7   Luton vs Sheff Utd      Luton   3-1   14/1     0.071429
8   Luton vs Sheff Utd  Sheff Utd   2-1   16/1     0.062500
9   Luton vs Sheff Utd       Draw   2-2   16/1     0.062500
10  Luton vs Sheff Utd  Sheff Utd   2-0   25/1     0.040000
11  Luton vs Sheff Utd      Luton   3-2   25/1     0.040000
12  Luton vs Sheff Utd  Sheff Utd   3-1   40/1     0.025000
13  Luton vs Sheff Utd  Sheff Utd   3-2   40/1     0.025000
14  Luton vs Sheff Utd       Draw   3-3   50/1     0.020000
15  Luton vs Sheff Utd  Sheff Utd   3-0 

### Testing the parsing function
Parameters:
filename (`str`): The path to the `HTML file` containing football match statistics.

Returns:
pandas.DataFrame: A DataFrame containing parsed football match statistics, including team names, scores, odds, probabilities, and sorted by probabilities in descending order.

In [2]:
stats = parse_football_stats('Matches 060224/wolverhampton_brentford.html')
stats

,Game,Team,Score,Odds,Probability
0,Wolverhampton vs Brentford,Draw,1-1,13/2,0.153846
1,Wolverhampton vs Brentford,Wolverhampton,1-0,9/1,0.111111
2,Wolverhampton vs Brentford,Wolverhampton,2-1,9/1,0.111111
3,Wolverhampton vs Brentford,Wolverhampton,2-0,11/1,0.090909
4,Wolverhampton vs Brentford,Draw,0-0,11/1,0.090909
5,Wolverhampton vs Brentford,Brentford,2-1,12/1,0.083333
6,Wolverhampton vs Brentford,Brentford,1-0,12/1,0.083333
7,Wolverhampton vs Brentford,Draw,2-2,14/1,0.071429
8,Wolverhampton vs Brentford,Wolverhampton,3-1,18/1,0.055556
9,Wolverhampton vs Brentford,Brentford,2-0,18/1,0.055556


### Testing the Top N Outcomes function
Retrieve the top 'n_outcomes' outcomes from HTML files in the specified directory.

Parameters:
directory (`str`): The path to the directory containing `HTML files` with football match statistics.
n_outcomes (`int`, optional): The number of top outcomes to retrieve. Defaults to 2.

Returns:
pandas.DataFrame: A DataFrame containing the top 'n_outcomes' outcomes, including teams, scores, odds, and probabilities.

In [6]:
top_n_outcomes_from_folder('Matches 060224', 2)

,Game,Team,Score,Odds,Probability
0,Fulham vs Bournemouth,Draw,1-1,13/2,0.153846
1,Fulham vs Bournemouth,Fulham,2-1,9/1,0.111111
2,Wolverhampton vs Brentford,Draw,1-1,13/2,0.153846
3,Wolverhampton vs Brentford,Wolverhampton,1-0,9/1,0.111111
4,Tottenham vs Brighton,Draw,1-1,17/2,0.117647
5,Tottenham vs Brighton,Tottenham,2-1,9/1,0.111111
6,Nottm Forest vs Newcastle,Draw,1-1,7/1,0.142857
7,Nottm Forest vs Newcastle,Newcastle,2-1,17/2,0.117647
8,Luton vs Sheff Utd,Draw,1-1,7/1,0.142857
9,Luton vs Sheff Utd,Luton,1-0,8/1,0.125000
